In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series
from tkinter.messagebox import *
from tkinter import filedialog , messagebox
#from tkinter import *
import tkinter

In [21]:
def Message_Valide() :
    rep= messagebox.showinfo('Info', "Le fichier excel est créé...")
    TKI_Principal.destroy()
    
def FNC_Selection ( ) :
    global kfichier
    kfichier = filedialog.askopenfilename(initialdir = "/", title = "Selectionner le fichier à traiter")

def fichier():
    from openpyxl import load_workbook
    Del_annuelle=pd.ExcelFile(kfichier, engine='openpyxl')
    feuilles = Del_annuelle.sheet_names
    Deliberation = Del_annuelle.parse(feuilles[0])
    
    Deliberations = Deliberation
    Indice_colonne = np.array(Deliberations.iloc[9])
    Indice_colonne[len(Indice_colonne)-1] = "Decision"
    Deliberations = Deliberations.dropna(axis=0)
    Deliberations.columns = Indice_colonne
    Deliberations = Deliberations.reset_index(drop=True)
    #Deliberations.drop(0 , inplace=True)
    Deliberations.columns.values[len(Deliberations.columns)-2] = "TotalCredit"
    #Deliberations = Deliberations.reset_index(drop=True)
    Totalcredit = Deliberations.columns.values[len(Deliberations.columns)-2]
    Nom = Deliberations.columns.values[len(Deliberations.columns)-2]
    Prenom = Deliberations.columns.values[len(Deliberations.columns)-2]
    Deliberations=Deliberations.sort_values(by = [Totalcredit, Nom, Prenom], ascending = [False, True, True])
    
    # Determination du nombre d'etudiants en fonction de la deliberation
    repartion_etudiants = Deliberations.Decision.value_counts()
    
    pourcentage = np.array([])
    pourcentage_dec = np.array([])
    for m in repartion_etudiants:
        val = round(m*100/sum(repartion_etudiants),2)
        pourcentage = np.append(pourcentage,val)
        pourcentage_dec = np.append(pourcentage_dec,val%1)
    n = 0
    while sum(pourcentage) != 100:
        pourcentage_dec = np.sort(pourcentage_dec)
        pourcentage[n] = pourcentage[n] + pourcentage_dec[n] + 0.01
        n +=1
    pourcentage = np.append(pourcentage,100)
    pourcentage
    Decision = ["Passage_Definitive", "Passage_Conditionnel", "Redoublement", "Exclu", "Effectif_Total"]
    repartion1 = [repartion_etudiants[0], repartion_etudiants[2], repartion_etudiants[1], repartion_etudiants[3], sum(repartion_etudiants)]
    donnees = np.array([repartion1,pourcentage])
    donnees = np.transpose(donnees)
    column = ["Nombre_Etudiants", "Pourcentage"]
    df_list = {"pourcentage" : pourcentage, "Repartion" : repartion1}
    df_list=pd.DataFrame(donnees, index=Decision, columns = column)
    #Creation du Diagramme
    labels = df_list.index[:4]
    serie = df_list["Pourcentage"][:4]
    separation = (0.1, 0, 0, 0) # Séparation des tranches
    fig, ax = plt.subplots(figsize=(10,5))
    ax.pie(serie, explode=separation, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
    ax.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
    ax.set_title("Statistique de Délibération") # Titre de la figure
    
    # Ecriture spécifique.
    # Cette fonction créera le fichier s'il n'existe pas.
    Deliberation_UAM = pd.ExcelWriter("Resultat_UAM.xlsx", engine="xlsxwriter")

    ## Ecriture du DataFrame dans le fichier : "Deliberation_UAM_L1.xlsx"
    # Enregistrement du DataFrame "Deliberations" dans une feuille du fichier "Deliberation_UAM_L1.xlsx"
    Deliberations.to_excel(Deliberation_UAM, "Deliberation", index = False)
    # Enregistrement du DataFrame df_list dans une nouvelle feuille du fichier "Deliberation_UAM_L1.xlsx"
    df_list.to_excel(Deliberation_UAM, "Statistiques")
    # Enregistrement du résultat
    Deliberation_UAM.save()

    # Insertion du graphe dans la feuille "Statistiques"
    import xlwings as delib
    delibere = delib.Book("Resultat_UAM.xlsx")
    feuille = delibere.sheets("Statistiques")
    ax = ax.get_figure()
    feuille.pictures.add(ax, name="Statistiques", update=True)
    # Enregistrement du résultat
    Deliberation_UAM.save()
    plt.show()
    Message_Valide()


TKI_Principal = tkinter.Tk()
TKI_Principal.title("CREATION FICHIER DELIBERATION")
TKI_Principal.geometry("700x500")
TKI_Principal.config(background='#41B77F')
frame = tkinter.Frame(TKI_Principal, bg='#41B77F')
Label_title = tkinter.Label(frame, text="Bonjour!!", font=("courrier", 40), bg='#41B77F', fg='white')
Label_title.pack()#centrer le texte 
Label_sous_titre_select = tkinter.Label(frame, text="Cliquer pour selectionner le Fichier :", font=("courrier", 15), bg='#41B77F', fg='white')
Label_sous_titre_select.pack()
frame.pack(expand=tkinter.YES)# Centrer la boite
BUT_Selectionner = tkinter.Button ( frame , text = "Sélectionner le fichier excel ", font=("courrier", 20), bg='white', fg='#41B77F', command = FNC_Selection )
BUT_Selectionner.pack()

Label_sous_titre_creer = tkinter.Label(frame, text="Cliquer pour creer le Fichier de délibération :", font=("courrier", 15), bg='#41B77F', fg='white')
Label_sous_titre_creer.pack()
Boutton = tkinter.Button(frame, text="Créer le Fichier Excel", font=("courrier", 20), bg='white', fg='#41B77F', command=fichier)
Boutton.pack()
Boutton = tkinter.Button(frame, text= 'Quitter', font=("Helvatica", 20), bg='white', fg='#41B77F', command=TKI_Principal.destroy)
Boutton.pack()


TKI_Principal.mainloop ( )
